# Anomaly Detection Final

In [59]:
# Import libraries
!pip install pandas-gbq
!pip install shap
import shap
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from google.cloud import bigquery 
import pandas as pd 
import numpy as np
from sklearn.ensemble import IsolationForest 

In [60]:
# Initialize connection 
project = !gcloud config get-value project
project_id = project[0]
bq = bigquery.Client(project = project_id)

## Feature Engineering/Processing

In [61]:
# Read dataset in from Big Query -> To Do: Change data source for productionizing
dataset = 'ltaylor'
table = 'Cost_02_Copy_2'
query = f"""
SELECT * 
FROM `{project_id}.{dataset}.{table}`
"""
df = bq.query(query = query).to_dataframe()
df.head()

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


,invoice_date,service_description,sku_description,project_id,business_unit,cost
0,2023-10-19,Cloud Scheduler,Jobs,clgx-surveillance-prd-f609,None,0.002290
1,2023-10-19,Cloud Storage,Upload,clgx-surveillance-reg-ec2c,None,0.000000
2,2023-10-19,BigQuery,Analysis,clgx-surveillance-dr-66e6,None,0.885941
3,2023-10-19,BigQuery,Analysis,clgx-surveillance-reg-ec2c,None,0.002875
4,2023-10-19,BigQuery,Analysis,clgx-surveillance-np-reg-ccfe,None,0.742937


In [62]:
# Preprocessing

# Aggregatelower cost skus together as "Other"
skus =  ['Coldline Storage US Multi-region','Standard Storage US Multi-region', 'Active Logical Storage','Analysis',
                     'Long Term Logical Storage', 'Nearline Storage US Multi-region' , 'Archive Storage US Multi-region',
             'Archive Storage US Multi-region (Early Delete)','Archive Data Retrieval']
df['sku_description'] = [j if j in skus else 'Other' for j in df['sku_description']]

# Aggregate daily costs by sku description and project id
agg_df = df.groupby(['invoice_date','sku_description','project_id']).sum('cost').sort_values(['invoice_date','sku_description','project_id'])
agg_df = pd.DataFrame(agg_df).reset_index()
agg_df['invoice_date'] = agg_df['invoice_date'].astype(str)

# Create rolling median feature
date_sum = agg_df.groupby('invoice_date')['cost'].sum()
roll_med = date_sum.rolling(window=30, min_periods=1).median() 
# If cost greater than rolling median 1 else 0
agg_df['greater_than'] = [1 if (agg_df['cost'][i]) > roll_med[agg_df['invoice_date'][i]] and roll_med[agg_df['invoice_date'][i]] > 200 else 0 for i in range(len(agg_df))]

# Convert column to datetime
agg_df['invoice_date'] = pd.to_datetime(agg_df['invoice_date'])

# One-hot encode columns
enc_df = agg_df.copy()
enc_df = pd.get_dummies(enc_df)
enc_df.head()

# Make columns binary (1,0)
for i in enc_df.drop(columns = ['cost','invoice_date'], axis = 1).columns:
enc_df[i] = np.where(enc_df[i] == True, 1,0)
enc_df.head()





In [34]:
# Save invoice_dates
dates = enc_df['invoice_date']

# Drop invoice date column
enc_df.drop(labels = 'invoice_date', axis = 1, inplace = True)

# Shuffle dataset 
enc_df = enc_df.sample(frac = 1, random_state = 24)

In [35]:
# Keep features with high importance
enc_df = enc_df[['cost','greater_than', 'sku_description_Analysis', 'sku_description_Other']]

## Model Training

In [36]:
tune_model = IsolationForest(contamination = 0.0005,n_estimators = 150,max_samples = 0.8, 
                             max_features = 0.8,random_state = 24)
tune_model.fit(enc_df)
labels = tune_model.predict(enc_df)
prob_scores = abs(tune_model.score_samples(enc_df))

In [37]:
# Percentage of points classified as anomalous
df_res = enc_df.copy()
df_res['label'] = labels
df_res['label'].value_counts(normalize = True)

label
 1    0.999479
-1    0.000521
Name: proportion, dtype: float64

In [38]:
# Average and max confidence score
df_res['prob_score'] = prob_scores
df_res['invoice_date'] = dates
df_res = df_res.sort_values('invoice_date')
anomalies = df_res[df_res['label'] == -1].reset_index(drop = True)
print('Average Confidence Score: ',anomalies['prob_score'].mean())
print('Max Confidence Score: ',max(anomalies['prob_score']))

Average Confidence Score:  0.7713143413376942
Max Confidence Score:  0.8968610659248781


In [41]:
# Create df to visualize results
vis_df_agg = agg_df.copy()
vis_df_agg['prob_score'] = df_res['prob_score']
vis_df_agg['label'] = df_res['label']

In [ ]:
# Upload to BQ
vis_df_agg.to_gbq('ltaylor.vis_agg', 
                 project_id, 
                 chunksize=None, 
                 if_exists='replace')